In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from skimage import io, transform
from keras.utils import to_categorical
import time
import warnings
import shutil

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
from sklearn.model_selection import train_test_split
seed = 333
np.random.seed(seed)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from tensorflow import lite
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import random, os
import shutil
import matplotlib.pyplot as plt
from matplotlib.image import imread
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy, AUC
from sklearn.model_selection import train_test_split


!pip install tensorflow-addons==0.9.1
import tensorflow_addons
from tensorflow_addons.metrics import F1Score, CohenKappa


from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix

from tqdm import tqdm

In [ ]:
df = pd.read_csv('../input/retinoclasses/train (1).csv')

In [ ]:
#path to images
img_dir = "../input/retino/"

#list all available images type
print(os.listdir(img_dir))

In [ ]:
retinos = list(enumerate(os.listdir(img_dir)))

In [ ]:
retinos

In [ ]:
data = []
for x in retinos:
    data.append(x)

data

In [ ]:
#path to images
img_dir = "../input/retino/"
#list all available images type
DATA_DIR = os.listdir(img_dir)

In [ ]:
def load_data(img_dir):
    X = []
    y = []
    labels = []
    idx = 0
    for i,folder_name in data:
        if folder_name in DATA_DIR:
            labels.append(folder_name)
            for file_name in tqdm(os.listdir(f'{img_dir}/{folder_name}')):
                if file_name.endswith('png'):
                    im = io.imread(f'{img_dir}/{folder_name}/{file_name}')
                    if im is not None:
                        im = transform.resize(im, (100, 100))
                        X.append(im)
                        y.append(idx)

        idx+=1
    X = np.asarray(X)
    y = np.asarray(y)
    
    diagnosis_dict_binary = {
        0: 'No_DR',
        1: 'DR',
        2: 'DR',
        3: 'DR',
        4: 'DR'
    }

    diagnosis_dict = {
        0: 'No_DR',
        1: 'Mild',
        2: 'Moderate',
        3: 'Severe',
        4: 'Proliferate_DR',
    }


    df['b_type'] =  df['diagnosis'].map(diagnosis_dict_binary.get)
    df['type'] = df['diagnosis'].map(diagnosis_dict.get)    
    print(X)
    print(y)
    labels = np.asarray(labels)
    return X,y,labels

In [ ]:
X,y,labels = load_data(img_dir)

In [ ]:
pf=pd.read_csv('../input/select/csv_result-messidor_features.csv')

In [ ]:
pf.head()

In [ ]:
train_data, val = train_test_split(df, test_size = 0.15, stratify = df['type'])
train, test = train_test_split(train_data, test_size = 0.15 / (1 - 0.15), stratify = train_data['type'])




In [ ]:
global_dir = ''

train_dir = os.path.join(global_dir, 'train')
val_dir = os.path.join(global_dir, 'val')
test_dir = os.path.join(global_dir, 'test')

def subdirw():
    if os.path.exists(global_dir):
        shutil.rmtree(global_dir)

    if os.path.exists(train_dir):
        shutil.rmtree(train_dir)
    os.makedirs(train_dir)

    if os.path.exists(val_dir):
        shutil.rmtree(val_dir)
    os.makedirs(val_dir)

    if os.path.exists(test_dir):
        shutil.rmtree(test_dir)
    os.makedirs(test_dir)

In [ ]:
subdirw()

In [ ]:
main_dir = '../input/retino/'
for index, row in train.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['b_type']
    id_code = row['id_code'] + ".png"
    srcfile = os.path.join(main_dir, diagnosis, id_code)
    dstfile = os.path.join(train_dir, binary_diagnosis)
    os.makedirs(dstfile, exist_ok = True)
    shutil.copy(srcfile, dstfile)

for index, row in val.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['b_type']
    id_code = row['id_code'] + ".png"
    srcfile = os.path.join(main_dir, diagnosis, id_code)
    dstfile = os.path.join(val_dir, binary_diagnosis)
    os.makedirs(dstfile, exist_ok = True)
    shutil.copy(srcfile, dstfile)

for index, row in test.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['b_type']
    id_code = row['id_code'] + ".png"
    srcfile = os.path.join(main_dir, diagnosis, id_code)
    dstfile = os.path.join(test_dir, binary_diagnosis)
    os.makedirs(dstfile, exist_ok = True)
    shutil.copy(srcfile, dstfile)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)
train_img = X_train
train_labels = y_train
test_img = X_test
test_labels = y_test
train_img.shape, train_labels.shape, test_img.shape, test_labels.shape
rand_14 = np.random.randint(0, train_img.shape[0],28)
sample_img = train_img[rand_14]
sample_labels = train_labels[rand_14]
num_rows, num_cols = 2, 14
f, ax = plt.subplots(num_rows, num_cols, figsize=(12,5),gridspec_kw={'wspace':0.03, 'hspace':0.01})
train_path = 'train'
val_path = 'val'
test_path = 'test'

train_batches = ImageDataGenerator(rescale = 1./255).flow_from_directory(train_path, target_size=(224,224), shuffle = True)
val_batches = ImageDataGenerator(rescale = 1./255).flow_from_directory(val_path, target_size=(224,224), shuffle = True)
test_batches = ImageDataGenerator(rescale = 1./255).flow_from_directory(test_path, target_size=(224,224), shuffle = False)
for r in range(num_rows):
    for c in range(num_cols):
        image_index = r * 7 + c
        ax[r,c].axis("off")
        ax[r,c].imshow(sample_img[image_index])
        ax[r,c].set_title('%s' % sample_labels[image_index])
plt.show()
plt.close()

class Metrics(Callback):
    def __init__(self, xval, yval):
        super().__init__()
        self.xval = xval
        self.yval = yval
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val = self.xval
        y_val = self.yval
        
        y_pred = self.model.predict(X_val)
        y_pred = np.clip(y_pred,0,4)
        y_pred = y_pred.astype(int)

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return
    
kappa_metrics = Metrics(X_val, y_val)

In [ ]:
# a utility function that plots the losses and accuracies for training & validation sets across our epochs
def show_plots(history):
    loss_vals = history['loss']
    val_loss_vals = history['val_loss']
    epochs = range(1, len(history['acc'])+1)
    
    f, ax = plt.subplots(nrows=1,ncols=2,figsize=(16,4))
    
    # plot losses on ax[0]
    ax[0].plot(epochs, loss_vals, color='navy',marker='o', linestyle=' ', label='Training Loss')
    ax[0].plot(epochs, val_loss_vals, color='firebrick', marker='*', label='Validation Loss')
    ax[0].set_title('Training & Validation Loss')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].legend(loc='best')
    ax[0].grid(True)
    
    # plot accuracies
    acc_vals = history['acc']
    val_acc_vals = history['val_acc']

    ax[1].plot(epochs, acc_vals, color='navy', marker='o', ls=' ', label='Training Accuracy')
    ax[1].plot(epochs, val_acc_vals, color='firebrick', marker='*', label='Validation Accuracy')
    ax[1].set_title('Training & Validation Accuracy')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Accuracy')
    ax[1].legend(loc='best')
    ax[1].grid(True)
    
    plt.show()
    plt.close()
    
    # delete locals from heap before exiting
    del loss_vals, val_loss_vals, epochs, acc_vals, val_acc_vals

In [ ]:
def ensembleCNN():
    #create multiple cnn model for ensembling
    model = Sequential()

    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (100, 100, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))


    model.add(Conv2D(256, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))

    # use adam optimizer and categorical cross entropy cost
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    

In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(16, (3,3), padding="same", input_shape=(224,224,3), activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(32, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
 
    layers.Conv2D(32, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(1,1)),
    layers.BatchNormalization(),
 
    layers.Conv2D(64, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(1,1)),
    layers.BatchNormalization(),
    
    layers.Conv2D(128, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(128, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
     layers.Conv2D(128, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(1,1)),
    layers.BatchNormalization(),
 
    
    layers.Conv2D(256, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(256, (3,3), padding="same", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(1,1)),
    layers.BatchNormalization(),
    
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.15),
    layers.Dense(2, activation = 'softmax')
    
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['acc','AUC',tensorflow_addons.metrics.F1Score(num_classes=2, average='weighted'),tensorflow_addons.metrics.CohenKappa(num_classes=5)])

history = model.fit(train_batches,
                    epochs=15,
                    validation_data=val_batches)
                    


In [ ]:
model.save("my_model.h5")

In [ ]:
reconstructed_model = keras.models.load_model("my_model.h5")

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['val_auc'])
plt.plot(history.history['val_f1_score'])
#plt.plot(history.history['TruePositive'])
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val_acc', 'val_f1_score', 'acc', 'val_cohen_kappa'])

model.load_weights("my_model.h5")
y_val_pred = model.predict(train_img)
#clipping the value to range of 0-4, and round it to the nearest integer
y_val_pred = np.clip(y_val_pred,0,4).astype(int)

labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(df['diagnosis'].astype('int'), y_val_pred)
df_cm = pd.DataFrame(cnf_matrix, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, cmap="Blues")
plt.show()

model.summary()